In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-whitegrid")
import levenberg_marquardt
from matplotlib.pyplot import figure
import heston
from importlib import reload
reload(heston)
from heston import calibrate_heston, get_tick
import datetime
from scipy import stats as sps
from typing import Union
import h5py
import matplotlib.dates as mdates
from tqdm.notebook import tqdm

In [ ]:
# %%time
# raw = pd.HDFStore('../datasets/eth-20230127.h5','r').get("/eth")

In [3]:
def process_data(data):
    # only options
    df = data.copy()
    df = df[(df["instrument"].str.endswith("C")) | (df["instrument"].str.endswith("P"))].sort_values("dt")
    df["type"] = np.where(df["instrument"].str.endswith("C"), "call", "put")
    
    perpetuals = data[data["instrument"].str.endswith("PERPETUAL")][["dt", "price"]].copy()
    perpetuals = perpetuals.rename(columns = {"price": "underlying_price"}).sort_values("dt")
    
    def get_strike(x):
        return int(x.split("-")[2])
    
    def get_expiration(x):
        return x.split("-")[1]
    

    df["strike_price"] = df["instrument"].apply(get_strike)
    df["expiration"] = df["instrument"].apply(get_expiration)
    
    def unix_time_millis(dt):
        epoch = datetime.datetime.utcfromtimestamp(0)
        return int((dt - epoch).total_seconds() * 1000_000)
    
    def get_normal_date(s):
        """Function to convert date to find years to maturity"""
        monthToNum = {
            "JAN": 1,
            "FEB": 2,
            "MAR": 3,
            "APR": 4,
            "MAY": 5,
            "JUN": 6,
            "JUL": 7,
            "AUG": 8,
            "SEP": 9,
            "OCT": 10,
            "NOV": 11,
            "DEC": 12,
        }

        full_date = s.split("-")[1]
        try:
            day = int(full_date[:2])
            month = monthToNum[full_date[2:5]]
        except:
            day = int(full_date[:1])
            month = monthToNum[full_date[1:4]]
        
        year = int("20" + full_date[-2:])
        exp_date = datetime.datetime(year, month, day)
        return unix_time_millis(exp_date)
    
    df["dt"] = pd.to_datetime(df["dt"])
    perpetuals["dt"] = pd.to_datetime(perpetuals["dt"])
    
    df = pd.merge_asof(df, perpetuals, on="dt",
                       tolerance=pd.Timedelta('7 minutes'),
                       direction='nearest',)
    
    df["timestamp"] = df["dt"].apply(unix_time_millis)
    df["expiration"] = df["instrument"].apply(get_normal_date)
    df = df.rename(columns = {"price": "mark_price"})
    
    
    return df


In [ ]:
# %%time
# start = process_data(raw)


In [ ]:
# df = start.copy()

In [4]:
df = pd.read_csv(f"../datasets/deribit_options_chain_2022-12-01_OPTIONS.csv")
df = df.drop(columns = ["exchange", "open_interest", 
                    "last_price", "bid_price", "bid_amount", "bid_iv",
                           "ask_price", "ask_amount", "ask_iv", "underlying_index",
                           "delta", "gamma", "vega", "theta" ,"rho"])
currency = "ETH"
ind = [symbol.startswith(currency) for symbol in df.symbol]
df = df[ind].reset_index(drop=True)

# after expiration
df = df[df["timestamp"] > 1669881600000000]
# df = df[df["timestamp"]<=1669928620213347.0]

In [11]:
calibrated_params_nu0

NameError: name 'calibrated_params_nu0' is not defined

In [17]:
# calibrate base
number_of_dots = 30
timestamps_hm = []
names = ["kappa", "nu_bar", "sigma", "rho", "nu0"]
params_all = []
params_nu0 = []
errors_all = []
errors_nu0 = []

a = np.float64(3.0)  # kappa    a                 |  mean reversion rate | kappa (Article) | kappa (Roland's)
b = np.float64(0.1)  # v_infinity                |  long term variance | nu_bar           | alpha_bar 
c = np.float64(0.25)  # sigma                    |  volatility of volatility | sigma      | mu
rho = np.float64(0.01)  # rho                    |  correlation between spot and vol |rho | rho
v0 = np.float64(0.08) # init variance            | initial variance | nu_0                | alpha_0
start_params = np.array([a, b, c, rho, v0])
print("Optimize first params with random start once")
calibrated_params_all, first_error_all = calibrate_heston(df = df, 
                                                          timestamp= 1669913599869333.2,
                                                          start_params = start_params, calibration_type = "all")
calibrated_params_nu0, first_error_nu0 = calibrate_heston(df = df, 
                                                          timestamp= 1669913599869333.2,
                                                          start_params = start_params, calibration_type = "nu0")
print("all first calibrated", calibrated_params_all)
print("nu0 first calibrated", calibrated_params_nu0)

# Get ticks with similar distnce from available data
timestamps = np.linspace(df.timestamp.min(), df.timestamp.max(), number_of_dots)
for timestamp in tqdm(timestamps):
    try:
        # zero params for each point are params from last step
        calibrated_params_all, error_all = calibrate_heston(df = df, start_params = calibrated_params_all, 
                                                            timestamp = timestamp, calibration_type = "all")
        print("All calibrated:", calibrated_params_all)
        
        calibrated_params_nu0, error_nu0 = calibrate_heston(df = df, start_params = calibrated_params_nu0, 
                                                            timestamp = timestamp, calibration_type = "nu0")
        print("nu0 calibrated:", calibrated_params_nu0)

        # save all calib_params
        params_all.append(calibrated_params_all)
        errors_all.append(error_all)

        # save nu0 calib params
        params_nu0.append(calibrated_params_nu0)
        errors_nu0.append(error_nu0)
        
        # save timestame that could calibrate
        timestamps_hm.append(timestamp)
        
    except ValueError:
        # if could not calibrate, start next calibration with prev params
        print(f"Could not calibrate LM on {timestamp}")
        try:
            calibrated_params_all = params_all[-1]
            calibrated_params_nu0 = params_nu0[-1]
        except IndexError:
            calibrated_params_all = start_params
            calibrated_params_nu0 = calibrated_params_all
        
    except IndexError:
        print(f"Could not calibrate nu0 {timestamp}")
        try:
            calibrated_params_all = params_all[-1]
            calibrated_params_nu0 = params_nu0[-1]
        except IndexError:
            calibrated_params_all = start_params
            calibrated_params_nu0 = calibrated_params_all
        
    except ZeroDivisionError:
        print(f"Zero dev jacobian on {timestamp}")
        try:
            calibrated_params_all = params_all[-1]
            calibrated_params_nu0 = params_nu0[-1]
        except IndexError:
            calibrated_params_all = start_params
            calibrated_params_nu0 = calibrated_params_all
    


Optimize first params with random start once
all first calibrated [ 2.08791857  1.32906517  4.13569551 -0.36289758  0.6339262 ]
nu0 first calibrated [13.50791069  0.83299392  7.18593012 -0.37433675  0.43972661]


  0%|          | 0/30 [00:00<?, ?it/s]

All calibrated: nan
Could not calibrate nu0 1669881600006000.0
All calibrated: [ 2.32805613  1.28080411  4.33454607 -0.35871687  0.64446119]
Could not find jacobian on 1669883586204413.8
All calibrated: [ 2.5131332   1.241215    4.43814762 -0.358629    0.64452587]
Could not find jacobian on 1669885572402827.5
All calibrated: [ 2.34983373  1.27933238  4.35815659 -0.36140762  0.64224015]
nu0 calibrated: [18.11152982  0.80970155  9.13184157 -0.38743633  0.35700961]
All calibrated: [ 2.48282892  1.25571864  4.45838295 -0.36175477  0.64412621]
Could not calibrate LM on 1669889544799655.2
All calibrated: [ 2.38647779  1.26966803  4.39111308 -0.36125754  0.64053262]
Could not calibrate LM on 1669891530998069.0
All calibrated: [ 2.36608908  1.26911959  4.3457762  -0.36217395  0.64093761]
Could not calibrate LM on 1669893517196482.8
All calibrated: [ 2.38083265  1.2715396   4.38497299 -0.36241322  0.63992575]
Could not calibrate LM on 1669895503394896.5
All calibrated: [ 2.57787524  1.2323057  


KeyboardInterrupt



In [ ]:
# single_instr = df[df["instrument"] == df.sample(1).instrument.iloc[0]]
# plt.plot(single_instr["dt"], single_instr["mark_price"])
# plt.plot(df["dt"], df["underlying_price"])

In [ ]:
# timestamps_hm = [datetime.datetime.fromtimaestamp(x/1000000.0) for x in timestamps_hm]
# expirarions = df.expiration.unique()
# expirarions = [datetime.datetime.fromtimestamp(ms/1000000000.0) for ms in expirarions]
# expirarions
fig, axs =  plt.subplots(nrows=3, ncols=2, figsize=(20, 15))

axs[0, 0].plot(timestamps_hm, kappas_all)
axs[0, 0].plot(timestamps_hm, kappas_nu0)
axs[0, 0].set_title(r"$\kappa$ - mean reversion rate")

axs[0, 1].plot(timestamps_hm, nu_bars_all)
axs[0, 1].plot(timestamps_hm, nu_bars_nu0)
axs[0, 1].set_title(r"$\bar{\nu}$ - long term variance")

axs[1, 0].plot(timestamps_hm, sigmas_all, label = "all")
axs[1, 0].plot(timestamps_hm, sigmas_nu0, label = "nu0")
axs[1, 0].set_title(r"$\sigma$ - vol of vol")

axs[1, 1].plot(timestamps_hm, rhos_all)
axs[1, 1].plot(timestamps_hm, rhos_nu0)
axs[1, 1].set_title("ρ -rho")

axs[2, 0].plot(timestamps_hm, nu0s_all)
axs[2, 0].plot(timestamps_hm, nu0s_nu0)
axs[2, 0].set_title(r"$\nu_0$ - initial variance")

axs[2, 1].plot(timestamps_hm, errors_all)
axs[2, 1].plot(timestamps_hm, errors_nu0)
axs[2, 1].set_title("error")



for i in range(3):
    for j in range(2):
        for exp in expirarions:
#             axs[i, j].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
#             axs[i, j].axvline(x=exp, color = "black", linestyle  = "--")
            if i == j == 0:
#                 axs[i, j].axvline(x=exp, color = "black", linestyle  = "--", label = exp)
                pass


plt.figlegend() 
fig.suptitle("Heston params dynamic")

plt.show()

In [6]:
np.linspace(df.timestamp.min(), df.timestamp.max(), 10)[5]

1669913599869333.2